# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21686515


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/32 [00:00<?, ?it/s]

Rendering models:   6%|▋         | 2/32 [00:05<01:28,  2.95s/it]

Rendering models:   9%|▉         | 3/32 [00:06<01:05,  2.27s/it]

Rendering models:  12%|█▎        | 4/32 [00:08<01:04,  2.30s/it]

Rendering models:  16%|█▌        | 5/32 [00:10<00:56,  2.08s/it]

...

Rendering models:  75%|███████▌  | 24/32 [00:24<00:04,  1.62it/s]

Rendering models:  78%|███████▊  | 25/32 [00:25<00:04,  1.68it/s]

Rendering models:  84%|████████▍ | 27/32 [00:25<00:02,  2.15it/s]

Rendering models:  88%|████████▊ | 28/32 [00:26<00:02,  1.44it/s]

not-logged-in-5e3990f07025a175033b    0.000429
ctiborekskutr                         0.000481
asucich                               0.000477
shocko61                              0.000419
not-logged-in-f7e2260ed4a9517a3e4a    0.005682
bob_the_one                           0.226175
rnevils                               0.000422
kcarte13                              0.000314
nsyee                                 0.000441
justsomenoodles                       0.000509
not-logged-in-e4d3020e215c78c076c6    0.074736
ericmalani                            0.000570
danielrwik                            0.000423
not-logged-in-4cd99e50bbc7058bf4e8    0.000468
stansall                              0.001196
surfgimp                              0.000473
Lavadude                              0.038327
jordanrushworth                       0.000636
djsimister                            0.000417
BigboiLuna2023                        0.035045
jmacd297                              0.000649
not-logged-in

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())